<a href="https://colab.research.google.com/github/Yeyeong99/Aiffel/blob/main/contest/%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C_%EA%B3%A0%EA%B0%9D_%EC%84%B8%EA%B7%B8%EB%A8%BC%ED%8A%B8_%EB%B6%84%EB%A5%98/RadomforestClassifier_%ED%99%9C%EC%9A%A9_%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C_%EA%B3%A0%EA%B0%9D_%EC%84%B8%EA%B7%B8%EB%A8%BC%ED%8A%B8_%EB%B6%84%EB%A5%98_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import

In [1]:
import pandas as pd
import numpy as np
import gc

from sklearn.preprocessing import LabelEncoder

### Data Load

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_df = pd.read_csv("/content/drive/MyDrive/train_data.csv")
test_df = pd.read_csv("/content/drive/MyDrive/test_data.csv")

<ipython-input-3-9b25485309a6>:1: DtypeWarning: Columns (299,384) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("/content/drive/MyDrive/train_data.csv")
<ipython-input-3-9b25485309a6>:2: DtypeWarning: Columns (298) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("/content/drive/MyDrive/test_data.csv")


### Data Preprocessing(2) : Encoding

In [4]:
feature_cols = [col for col in train_df.columns if col not in ["ID", "Segment"]]

X = train_df[feature_cols].copy()
y = train_df["Segment"].copy()

# 타깃 라벨 인코딩
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

In [6]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

X_test = test_df.copy()

encoders = {}  # 각 컬럼별 encoder 저장

for col in categorical_features:
    # Convert the column to string type before applying LabelEncoder
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

    le_train = LabelEncoder()
    X[col] = le_train.fit_transform(X[col])
    encoders[col] = le_train
    unseen_labels_val = set(X_test[col]) - set(le_train.classes_)
    if unseen_labels_val:
        le_train.classes_ = np.append(le_train.classes_, list(unseen_labels_val))
    X_test[col] = le_train.transform(X_test[col])

In [7]:
gc.collect()

15

### Train / Validation Split

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42
)

### Train

### Predict

### Submission

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [12]:
param_grid = {
    'n_estimators': [50, 100, 200],  # 트리 개수
    'max_depth': [None, 10, 20],     # 최대 깊이
    'min_samples_split': [2, 5, 10], # 노드 분할 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],   # 리프 노드 최소 샘플 수
    'bootstrap': [True, False]       # 부트스트래핑 여부
}

In [13]:
rf_model = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    n_jobs=-1,
    max_features="sqrt",
    random_state=42,
)

In [14]:
# GridSearchCV 적용
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=5,         # 5-Fold 교차 검증
    n_jobs=-1,    # 모든 CPU 코어 사용
    verbose=2,    # 진행 과정 출력
    scoring='accuracy'  # 평가 기준
)

### 모델 저장하기, 학습 중 메시지 확인

In [ ]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


In [ ]:
print("Best parameters:", grid_search.best_params_)

In [ ]:
best_rf = grid_search.best_estimator_

In [ ]:
X_test.drop(columns=['ID'],inplace=True)

In [ ]:
y_test_pred_rf = best_rf.predict(X_test)
y_test_pred_rf_labels = le_target.inverse_transform(y_test_pred_rf)


In [ ]:
test_data_rf = test_df.copy()  # 원본 유지

In [ ]:
test_data_rf["pred_label"] = y_test_pred_rf_labels

In [ ]:
submission = test_data_rf.groupby("ID")["pred_label"] \
    .agg(lambda x: x.value_counts().idxmax()) \
    .reset_index()

submission.columns = ["ID", "Segment"]

submission.to_csv('/content/drive/MyDrive/creditcard_randomforest_gridsearch.csv',index=False)